In [15]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [16]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
test_df = pd.read_csv ('Dataset/testset.csv')

data_test = test_df[['x_test','future']].to_numpy()

x_test_string = data_test[:,0]
y_test_string = data_test[:,1]
x_test = []
y_test = []
for i in x_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_test = [int(j) for j in i.split()]
    x_test.append(input_x_test)

for i in y_test_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_test = [int(j) for j in i.split()]
    y_test.append(input_y_test)
x_test = np.array(x_test)
y_test = np.array(y_test)
    


notes_ = notes_df.to_numpy()[:,1]
unique_notes = dict(enumerate(notes_.flatten(), 0))
# unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}


In [17]:
from MusicDataset import *
test_set = MusicDataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

In [18]:
from Models import Wavenet,LSTM
Net = torch.load('Trained_Model/model.pth')
print(Net)

Sequence(
  (embedding): Embedding(173, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (softmax): Softmax(dim=None)
  (linear3): Linear(in_features=128, out_features=173, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [19]:
total_preds = 0
correct_preds = 0
future_preds = 8
for i, data in enumerate(testloader, 0):

    input , label = data
    cumm_output = torch.zeros(0,len(unique_notes)).to(device)
    cumm_label  = np.array([],dtype=int)
    for k in range(future_preds):
        output = Net(input.to(device),input.shape[0])
        cumm_output = torch.cat((cumm_output,output))
        cumm_label = np.concatenate((cumm_label,label[:,k]))
        next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label[:,k].to(device))
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
                                               for ind,j in enumerate(input)])[:,1:]) 
test_acc =  float(correct_preds)/float(total_preds) *100 
testreport ="Testing Accuracy : \n correct predictions  : {} \n total predictions : {} \n Testing Accuracy : {} \n ------------------------\n".format(correct_preds,total_preds,test_acc)
print(testreport)  

KeyboardInterrupt: 

In [6]:
#######################Errrrrrrror
# # Time to generate some Music!!!
# import random
# from Dataset import midi_helper 
# for j in range(10):
#     index = random.randint(0,len(x_test))
#     print("Taking the seed tune as follows:")
#     print(x_test[index])
#     tune = x_test[index]
#     input = np.empty((1,32),dtype=int)
#     input[0] = tune
#     input = torch.from_numpy(input)
#     next_preds = 64
#     for i in range(next_preds):
#         output = Net(input.to(device),input.shape[0])
#         next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
#         input = input.cpu().detach().numpy()
#         input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                    for ind,j in enumerate(input)])[:,1:]) 

#         tune = np.insert(tune,-1,next_preds[0])
#     tune = [unique_notes[i] for i in tune]
#     path = './Outputs/music'+str(j)+'.midi'
#     midi_helper.convert_to_midi(tune,path)


In [20]:
# Time to generate some Music!!!
import random
from Dataset import midi_helper 

# Assuming convert_to_midi() function only takes one argument
for j in range(10):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device), input.shape[0])
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])
        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
    path = './Outputs/music1'+str(j)+'.midi'
    midi_helper.convert_to_midi(tune)  # Remove the path argument


Taking the seed tune as follows:
[ 71 126 108 105 105  25 171 171 171  14 171 114 171  65 109 159 109 159
 109  98 116 125  32 125  56 129  56  89  16  89 129  89]
Taking the seed tune as follows:
[144  76 164 100  76 131 160  52  46   4  16  11  76  76  50 103 119 167
  56 103 119 167  56 103 119 167  56 103 119 167  56 103]
Taking the seed tune as follows:
[146 171 106 162   4 146 106 162  96   8 146 171 162  19 146 171 162  19
 146 171 162  19 146 171 162  19 106 162 130 167 106 162]
Taking the seed tune as follows:
[ 16  98 109   6  10  97 163 146  10   9 115 111  34   6  45 109  29  98
  11 125   4 171 108 117  43 172 112 143  13  51   0 141]
Taking the seed tune as follows:
[ 97 106  62  29 129  51  62 155  97 140  62 112  97 140  62 112  97 135
  62 117  97 135  62 117  97   4  62 145  97   4  62 145]
Taking the seed tune as follows:
[129  68  70   2  19  71  73   2  71  73   2  72   2   2   2   2   2  94
 131  94  94  94  94  94  80  58 157  80 157  80  49 157]
Taking the seed 

In [8]:
import os
import random
import numpy as np
import torch
from Dataset import midi_helper 

# Assuming convert_to_midi() function only takes one argument
output_directory = './Outputs/'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

for j in range(10):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device), input.shape[0])
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])
        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
    filename = f'music{j}.midi'
    path = os.path.join(output_directory, filename)  # Get the full path
    print("Saving file at:", path)  # Display the full path
    midi_helper.convert_to_midi(tune, path)  # Provide the full path argument


Taking the seed tune as follows:
[ 38  79  53  78  78  74  44  78  78  91 112 112  91 140 140  74 112 112
  74 140 140  83  83 112 112  83  83 140 140  17  69  91]
Saving file at: ./Outputs/music0.midi


TypeError: convert_to_midi() takes 1 positional argument but 2 were given

In [10]:
#pip install music21


In [39]:
import os
import random
import numpy as np
import torch
from Dataset import midi_helper 

# Assuming convert_to_midi() function only takes one argument
output_directory = './Outpu/'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

for j in range(5):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device), input.shape[0])
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])
        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
#     filename = f'music{j}.midi'
#     path = os.path.join(output_directory, filename)  # Get the full path
#     print("Saving file at:", path)  # Display the full path
    midi_helper.convert_to_midi(tune)  # Remove the path argument



Taking the seed tune as follows:
[145  11 145 145 163  69 135 163  69  91  17 163  69  91  17 163  69  36
  98   2 103   2   2   2  98   2 149 140  39  91  95 130]
Taking the seed tune as follows:
[155  34   0 130 155   0 120 108  88 120  38 172 111 151 120 117 128 141
 130   0  51  29  13  51 128   4 141 128 151  43 115 120]
Taking the seed tune as follows:
[ 11   6 163 111 163   6 163  11  39 163  11   6  29 163  11 111   4 163
 108 162  43 163 117 162  11 111   4  97 111  97  34  97]
Taking the seed tune as follows:
[109 135 146   4   8  34 109 135 146 130 125  34 109 108 167  34 116  43
  31  61  52  47  25 116 116   9   9  77  33 116  25 116]
Taking the seed tune as follows:
[ 84  67  67  44 171 165  23 165  99 165  99  32 134  26  72 134  48  66
  26 134  48   6  63 154  72  26  85  48  63  72 154  52]


In [ ]:
from IPython.display import Audio
from pretty_midi import PrettyMIDI

In [42]:
#pip install midi2audio

In [1]:
from midi2audio import FluidSynth

#Play MIDI
fs = FluidSynth()
fs.midi_to_audio('music.mid', 'output.wav')

ModuleNotFoundError: No module named 'midi2audio'

In [45]:
#import pygame

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [54]:
#pip install MIDIUtil

In [52]:
# pip install mido==1.2.9


In [56]:
#pip install musicpy

In [57]:
from musicpy import *


In [51]:
# import mido
# mid = mido.MidiFile('music.mid')
# for msg in mid.play():
#     port.send(msg)

In [61]:
#lay("music.mid")

In [63]:
import os
import random
import numpy as np
import torch
from Dataset import midi_helper 

# Assuming convert_to_midi() function only takes one argument
output_directory = './Outpu/'
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

for j in range(10):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = Net(input.to(device), input.shape[0])
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind]) 
                                           for ind, j in enumerate(input)])[:, 1:])
        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
#     filename = f'music{j}.midi'
#     path = os.path.join(output_directory, filename)  # Get the full path
#     print("Saving file at:", path)  # Display the full path
    midi_helper.convert_to_midi(tune)  # Remove the path argument



Taking the seed tune as follows:
[157  94  85  11 135   8 117  85 135 103 172 147 162  94 147 162  94 145
  11 162  11  94 145  11 162 130 103  11 145  17 162  94]
Taking the seed tune as follows:
[ 34 109  45   6  29  97  62  32  29  97 115 125 130 167  65  42  65  42
  65  42  65  42 130 153 167  60   9   9  34 109  45   6]
Taking the seed tune as follows:
[106  32 117  91  73 117   2 113  84   2  71  17 161  17 123  17 161  71
  84  36  71  17 102 123  17 102 123   8 103   8 103 140]
Taking the seed tune as follows:


IndexError: index 8014 is out of bounds for axis 0 with size 8014

In [64]:
old_filename = r'music.mid'

In [65]:
old_filename


'C:\\Users\\Admin\\Python\\Music_generation\\Music-Gen-AI-main\\music'